In [1]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='your_api_key_here', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://job-boards.greenhouse.io/grammarly/jobs/6916911")
page_data = loader.load().pop().page_content
print(page_data)

Job Application for Data Scientist at GrammarlyBack to jobsData ScientistBerlin; HybridApplyGrammarly team members in this role must be based in Berlin or Kyiv, and they must be able to collaborate in person at least two days per week
About Grammarly
Grammarly is the trusted AI assistant for communication and productivity, helping over 40 million people and 50,000 organizations do their best work. Companies like Atlassian, Databricks, and Zoom rely on Grammarly to brainstorm, compose, and enhance communication that moves work forward. Grammarly works where you work, integrating seamlessly with over 500,000 applications and websites. Founded in 2009, Grammarly is No. 7 on the Forbes Cloud 100, one of TIME’s 100 Most Influential Companies, one of Fast Company’s Most Innovative Companies in AI, and one of Inc.’s Best Workplaces.
The Opportunity
To achieve our ambitious goals, we’re looking for a Data Scientist to join our growing Data team and community. Data scientists on our team work c

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Data Scientist',
 'experience': '5+ years of work experience in Data Science',
 'skills': ['Statistical modeling',
  'Causal inference',
  'A/B testing',
  'Offline evaluations',
  'Metric validation',
  'Regression',
  'Classification',
  'SQL',
  'Python',
  'PySpark'],
 'description': 'To achieve our ambitious goals, we’re looking for a Data Scientist to join our growing Data team and community. Data scientists on our team work closely with cross-functional partners in Product and Engineering to define and improve Grammarly’s core product experience.'}

In [23]:
import pandas as pd

df = pd.read_csv("portfolio_roles.csv")
df

,role,experience,skills,description,links
0,Data Scientist,5+ years of work experience in Data Science,"Statistical modeling, Causal inference, A/B te...","To achieve our ambitious goals, we’re looking ...",https://example.com/portfolio/data-scientist
1,Backend Developer,4+ years of experience in backend systems and ...,"Node.js, Express.js, MongoDB, PostgreSQL, REST...",We’re seeking a Backend Developer to design sc...,https://example.com/portfolio/backend-developer
2,Frontend Developer,3+ years of frontend development experience,"React, Redux, JavaScript, HTML5, CSS3, TypeScr...",We are looking for a Frontend Developer to bui...,https://example.com/portfolio/frontend-developer
3,DevOps Engineer,5+ years working in cloud infrastructure and a...,"AWS, Terraform, Kubernetes, Jenkins, CI/CD Pip...",Join our DevOps team to improve system reliabi...,https://example.com/portfolio/devops-engineer
4,Machine Learning Engineer,4+ years of experience in machine learning dev...,"Python, TensorFlow, PyTorch, scikit-learn, MLf...",We’re hiring an ML Engineer to deploy and opti...,https://example.com/portfolio/machine-learning...
5,Cybersecurity Engineer,5+ years of experience in securing infrastruct...,"Network Security, OWASP Top 10, Penetration Te...",We’re looking for a Cybersecurity Engineer to ...,https://example.com/portfolio/cybersecurity-en...
6,Cloud Engineer,4+ years working with cloud services and infra...,"Azure, AWS, Google Cloud Platform, Terraform, ...","Looking for a Cloud Engineer to design, deploy...",https://example.com/portfolio/cloud-engineer
7,Mobile Developer,3+ years developing Android/iOS apps,"Flutter, React Native, Swift, Kotlin, Dart, Ap...","As a Mobile Developer, you’ll lead the develop...",https://example.com/portfolio/mobile-developer
8,AI Researcher,6+ years of experience in AI research or acade...,"Deep Learning, Reinforcement Learning, NLP, Tr...",Join our AI Research Lab to publish cutting-ed...,https://example.com/portfolio/ai-researcher
9,Full Stack Developer,5+ years of experience across frontend and bac...,"JavaScript, React, Node.js, SQL, MongoDB, Grap...",We’re hiring a Full Stack Developer to lead en...,https://example.com/portfolio/full-stack-devel...


In [24]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["skills"],
                       metadatas={"links": row["links"]},
                       ids=[str(uuid.uuid4())])

In [30]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'l

In [28]:
job = json_res
job['skills']

['Statistical modeling',
 'Causal inference',
 'A/B testing',
 'Offline evaluations',
 'Metric validation',
 'Regression',
 'Classification',
 'SQL',
 'Python',
 'PySpark']

In [33]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Arjun Reddy, a business development executive at Beboom. Beboom is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Beboom's portfolio: {link_list}
        Remember you are Arjun Reddy, BDE at Bebbom. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Science Solutions for Grammarly's Ambitious Goals

Dear Hiring Manager,

I came across the job description for a Data Scientist at Grammarly, and I'm excited to introduce Beboom, an AI & Software Consulting company that can help you achieve your ambitious goals. With our expertise in data science and software development, we can support your growing Data team and community.

Our team at Beboom has extensive experience in statistical modeling, causal inference, A/B testing, offline evaluations, metric validation, regression, and classification, which aligns perfectly with the skills required for the Data Scientist role. We're proficient in SQL, Python, and PySpark, ensuring seamless integration with your existing infrastructure.

To demonstrate our capabilities, I'd like to share some relevant examples from our portfolio:

* https://example.com/ml-python-portfolio (Machine Learning and Python solutions)
* https://example.com/ml-python-portfolio (Data Science and Pyt